In [42]:
from collections import Counter
import funcy
from fuzzywuzzy import fuzz
import numpy as np 
import pandas as pd 
import os

from data_cleaning_functions import (trans_remov_punc, standardize_whitespace, remove_punc, remove_suffix_from_last_name,
                                     clean_names, has_award, has_suffix, get_suffix, replace_last_name, 
                                     is_year_range, str_sim, clean_med_school, clean_std_college_name, long_form_date, 
                                    correct_mispellings)

from dev import (
    APP_DATA_DIR, GRANT_DATA_DIR,  NAME_COLS, RAW_NAME_COLS, FEMALE_FIRST_NAMES, FEMALE_MIDDLE_NAMES,
    RAW_CARD_ID, RAW_INDEX_IDS, PERSON_APPLICATION_ID, PERSON_ID, NIH_ID, PICKLE_DIR, STD_DIR)

from merging_functions import *

OUTPUT_CSV = False 

PERSONAL_INFO = [
    'clean_first_name', 'clean_last_name', 'clean_middle_name',
    'date_of_birth', 'medical_school', 'clean_college_trans']


# load autoreload extension
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
apps = pd.read_csv(os.path.join(APP_DATA_DIR, 'NIH_AAMC_index_cards_standardized.csv')).drop('Unnamed: 0', axis=1)

In [44]:
grant_data = pd.read_stata(os.path.join(GRANT_DATA_DIR, 'grant_data.dta'))

In [45]:
stars_data = pd.read_stata(os.path.join(GRANT_DATA_DIR, 'stars_data.dta'))

In [46]:
grant_data.columns = map(lambda x: x.lower(), grant_data.columns)

In [47]:
stars_data.columns = map(lambda x: x.lower(), stars_data.columns)

In [48]:
stars_data['is_star'] = 1

In [49]:
stars_data.head()

,newsetnb,fullname,first,middle,last,suffix,star_yob,star_yod,deg,mdyear,phdyear,exit,main_dept,is_star
0,S2700910,Philip Leder,Philip,,Leder,,1934.0,NaN,MD,1960,NaN,0,Genetics,1
1,A4700231,Robert P. Hebbel,Robert,P.,Hebbel,,1947.0,NaN,MD,1973,NaN,0,Internal Medicine,1
2,A6701706,Alfred G. Gilman,Alfred,G.,Gilman,,1941.0,2015.0,MD/PhD,1969,1969.0,1,Pharmacology,1
3,A4800150,"Paul A. Bunn, Jr.",Paul,A.,Bunn,Jr.,1945.0,NaN,MD,1971,NaN,0,Internal Medicine,1
4,A9001316,"Vincent T. DeVita, Jr.",Vincent,T.,DeVita,Jr.,1935.0,NaN,MD,1961,NaN,0,Internal Medicine,1


In [50]:
unique_grants = grant_data.sort_values('year').drop_duplicates(
    ['newsetnb'], keep='first')[
        ['first', 'middle', 'last', 'suffix', 'newsetnb', 'mdyear', 'phdyear', 'year']].rename(columns={'year': 'first_grant_year'})

In [51]:
unique_grants2 = pd.concat([stars_data.drop(['fullname'], axis=1), unique_grants], axis=0)

In [52]:
unique_grants2['clean_first_name'] = unique_grants2['first'].apply(clean_names)
unique_grants2['clean_middle_name'] = unique_grants2['middle'].apply(clean_names)

unique_grants2['clean_last_name'] = unique_grants2['last'].apply(clean_names)

In [53]:
female_mask = (
    (unique_grants2['clean_first_name'].isin(FEMALE_FIRST_NAMES))| (
        unique_grants2['clean_middle_name'].isin(FEMALE_MIDDLE_NAMES)))
unique_grants2['is_female'] = 0
unique_grants2.loc[female_mask, 'is_female'] = 1

In [54]:
def get_initial(raw_str):
    if pd.isnull(raw_str):
        return np.nan
    if len(raw_str) == 1:
        return raw_str
    return raw_str[0]

unique_grants2['clean_middle_initial'] = unique_grants2.clean_middle_name.apply(get_initial)
unique_grants2['clean_first_initial'] = unique_grants2.clean_first_name.apply(get_initial)

In [55]:
unique_grants3 = unique_grants2[unique_grants2.is_female==0].sort_values(['newsetnb', 'is_star'], ascending=False).drop_duplicates(['newsetnb'], keep='first')

In [56]:
apps_counter = Counter(apps.clean_last_name.values)
unique_grants3['last_name_counts'] = unique_grants3.clean_last_name.apply(lambda x: apps_counter[x])
unique_grants4 = unique_grants3[
    ((unique_grants3.last_name_counts>1) & pd.isnull(unique_grants3.mdyear)) | 
    ((unique_grants3.last_name_counts>1) & (unique_grants3.mdyear<1975) & (unique_grants3.mdyear>1960))]

In [57]:
unique_grants4 = unique_grants4.rename(columns={'mdyear': 'medschool_year_grad', 'phdyear': 'phd_year_grad', 
                                               'star_yob':'birth_year', 'star_yod': 'death_year'})

In [59]:
unique_grants4.shape

(24609, 21)

In [60]:
exact = pd.merge(
    left=apps, right=unique_grants4, on=['clean_first_name', 'clean_last_name'], how='inner', suffixes=['_x', '_y'])
exact.shape

(2682, 128)

In [61]:
print apps.shape

(3743, 109)


In [62]:
exact['grad_sim'] = exact.apply(lambda x: abs(x['medschool_year_grad_x'] - x['medschool_year_grad_y']), axis=1)
exact['activity_year_sim'] = exact.apply(lambda x: x['first_grant_year'] - x['medschool_year_grad_x'], axis=1)

def check_match(row):
    if row['grad_sim'] > 5:
        return 0
    # first grant shouldn't be given prior to med school graduation
    if row['activity_year_sim'] < 0:
        return 0
    if not pd.isnull(row['clean_middle_initial_sim']) and row['clean_middle_initial_sim'] < 40:
        return 0
    if row['last_name_counts'] == 1:
        return 1
    return 1

feature_dict = {
    'clean_middle_name': get_name_str_sim,
    'clean_middle_initial': get_name_str_sim,
} 

exact2 = add_similarity_features(exact, feature_dict, check_match, suffixes=['_x', '_y'])

print unique_grants4.is_star.sum()
print exact2[exact2['is_match']==1].is_star.sum()
print exact2.is_star.sum()

405.0
167.0
193.0


In [63]:
# change raw sim columns to 100- so can be sorted with small values first
exact2.loc[:, 'clean_middle_name_sim'] = 100 - exact2['clean_middle_name_sim']
exact2.loc[:, 'clean_middle_initial_sim'] = 100 - exact2['clean_middle_initial_sim']

In [64]:
# do sort in ascending, because grad year sim and activity sim should be as small as possible
sim_cols =  ['grad_sim', 'activity_year_sim', 'clean_middle_name_sim', 'clean_middle_initial_sim']
exact3 = filter_one_match_per_group_simple(exact2[exact2['is_match']==1], PERSON_ID, sim_cols, True)
exact4 = filter_one_match_per_group_simple(exact3, 'newsetnb', sim_cols, True)


In [65]:
exact5 = consolidate_merge_cols(exact4, ['_x', '_y'], [])

['clean_first_initial', 'clean_middle_initial', 'clean_middle_name', 'is_female', 'medschool_year_grad', 'birth_year']


In [66]:
nm_apps = get_nonmatched(apps, exact4[PERSON_ID].values, PERSON_ID)
nm_grant = get_nonmatched(unique_grants4, exact4['newsetnb'].values, 'newsetnb')

(3150, 109)
(24016, 21)


In [67]:
nm_apps_c = Counter(nm_apps.clean_last_name.values)
nm_grants_c = Counter(nm_grant.clean_last_name.values)
nm_apps.loc[:, 'last_name_counts'] = nm_apps.clean_last_name.apply(lambda x: nm_apps_c[x])

nm_grant.loc[:, 'last_name_counts'] = nm_grant.clean_last_name.apply(lambda x: nm_grants_c[x])

In [68]:
last_merge = pd.merge(left=nm_apps, right=nm_grant, how='inner', on=['clean_last_name'], suffixes=['_x', '_y'])
last_merge.shape

(67896, 130)

In [69]:
last_merge['grad_sim'] = last_merge.apply(lambda x: abs(x['medschool_year_grad_x'] - x['medschool_year_grad_y']), axis=1)
last_merge['activity_year_sim'] = last_merge.apply(lambda x: x['first_grant_year'] - x['medschool_year_grad_x'], axis=1)

def check_match(row):
    if row['grad_sim'] > 5:
        return 0
    # first grant shouldn't be given prior to med school graduation
    if row['activity_year_sim'] < 0 or row['activity_year_sim'] < 3:
        return 0
    if row['clean_first_initial_sim'] < 100:
        return 0
    if row['clean_middle_initial_sim'] < 100:
        return 0
    if row['clean_first_name_sim'] < 80:
        return 0
    if row['clean_middle_name_sim'] < 80:
        return 0
    return 1

feature_dict = {
    'clean_middle_name': get_name_str_sim,
    'clean_middle_initial': get_name_str_sim,
    'clean_first_name': get_name_str_sim,
    'clean_first_initial': get_name_str_sim,
} 

last_merge2 = add_similarity_features(last_merge, feature_dict, check_match, suffixes=['_x', '_y'])
print unique_grants4.is_star.sum()
print last_merge2[last_merge2['is_match']==1].is_star.sum()
print last_merge2.is_star.sum()

405.0
7.0
716.0


In [70]:
#filter to one match per id
last_merge2.loc[:, 'clean_first_name_sim'] = 100 - last_merge2['clean_first_name_sim']
last_merge2.loc[:, 'clean_middle_name_sim'] = 100 - last_merge2['clean_middle_name_sim']
last_merge2.loc[:, 'clean_middle_initial_sim'] = 100 - last_merge2['clean_middle_initial_sim']
last_merge2.loc[:, 'clean_first_initial_sim'] = 100 - last_merge2['clean_first_initial_sim']

In [71]:
sim_cols =  ['grad_sim', 'activity_year_sim', 'clean_first_name_sim', 'clean_first_initial_sim', 'clean_middle_name_sim', 'clean_middle_initial_sim']
last_merge3 = filter_one_match_per_group_simple(last_merge2[last_merge2['is_match']==1], PERSON_ID, sim_cols, True)
last_merge4 = filter_one_match_per_group_simple(last_merge3, 'newsetnb', sim_cols, True)

In [72]:
last_merge4.shape

(50, 139)

In [73]:
last_merge5 = consolidate_merge_cols(last_merge4, ['_x', '_y'], [])

['clean_first_initial', 'clean_first_name', 'clean_middle_initial', 'clean_middle_name', 'is_female', 'medschool_year_grad', 'birth_year', 'last_name_counts']


In [74]:
all_matches = pd.concat([last_merge5, exact5], axis=0)

In [75]:
nm_apps = get_nonmatched(apps, all_matches[PERSON_ID], PERSON_ID)

(3100, 109)


In [76]:
apps2 = pd.concat([all_matches, nm_apps], axis=0)

In [77]:
to_drop_cols = [c for c in apps2.columns if c.endswith('_missing') or c.endswith('_duplicate') or '_counts' in c
               or c.endswith('_sim')]
print to_drop_cols

['activity_year_sim', 'clean_first_initial_sim', 'clean_first_name_sim', 'clean_middle_initial_sim', 'clean_middle_name_sim', 'grad_sim', 'last_name_counts', 'newsetnb_duplicate', 'person_uuid_duplicate']


In [78]:
apps3 = apps2.drop(to_drop_cols+['is_match', 'first', 'last', 'middle', 'is_female', 'suffix'], axis=1)

In [82]:
apps3.loc[apps3.duplicated(NAME_COLS, keep=False), NAME_COLS+['grant_id', PERSON_ID, 'application_year', 'medical_school']]

,clean_first_name,clean_middle_name,clean_last_name,grant_id,person_uuid,application_year,medical_school
1951,JAMES,EDWARD,BROWN,NaN,1431.0,1966.0,YALE SCHOOL OF MEDICINE
1960,JAMES,EDWARD,BROWN,NaN,1407.0,1967.0,UNIVERSITY OF ROCHESTER SCHOOL OF MEDICINE AND...
1624,MARTIN,G,ALLEN,NaN,2263.0,1965.0,NaN
1625,MARTIN,G,ALLEN,NaN,2262.0,1965.0,NEW YORK UNIVERSITY SCHOOL OF MEDICINE
3213,DANIEL,MARTIN,BERKOWITZ,NaN,564.0,1969.0,YALE SCHOOL OF MEDICINE
3245,DANIEL,MARTIN,BERKOWITZ,NaN,529.0,1967.0,NEW YORK UNIVERSITY SCHOOL OF MEDICINE


In [83]:
has_birth_dt = ~pd.isnull(apps3.date_of_birth)
apps3.loc[has_birth_dt, 'birth_dt'] =  apps3.loc[has_birth_dt, 'date_of_birth']
apps3.loc[has_birth_dt, 'birth_year'] =  apps3.loc[has_birth_dt, 'date_of_birth'].apply(lambda x: pd.to_datetime(x).year)
apps4 = apps3.drop('date_of_birth', axis=1).rename(columns={'birth_dt': 'birth_date'})


In [84]:
apps4.to_csv(os.path.join(APP_DATA_DIR, 'NIH_AAMC_index_cards_grant_standardized.csv'))